In [ ]:
!pip install supervision

In [ ]:
import matplotlib.pyplot as plt
import supervision as sv
from pathlib import Path
import random

# Update the dataset short name and directory
dataset_shortname = "brackishmot"
data_dir = Path("/mnt/data/tmp/") / dataset_shortname
data_dir.mkdir(exist_ok=True, parents=True)

# Use the Kaggle API to download and unzip the dataset
# (Make sure the Kaggle API is installed and your credentials are configured)
!kaggle datasets download -d maltepedersen/brackishmot -p {data_dir} --unzip


In [ ]:
import os
import json
import pandas as pd
from pathlib import Path
from PIL import Image

# Define paths and parameters
dataset_root = data_dir / "BrackishMOT"
split = "train"
sequences = ['brackishMOT-97','brackishMOT-22','brackishMOT-70',
             'brackishMOT-07','brackishMOT-53','brackishMOT-42']
labels = ['fish', 'crab', 'shrimp', 'starfish', 'small fish', 'jellyfish']

# Initialize COCO dictionary
coco_dict = {"images": [], "annotations": [], "categories": []}

# Create categories (assuming class IDs are 1-indexed)
for i, label in enumerate(labels):
    coco_dict["categories"].append({
        "id": i + 1,
        "name": label,
        "supercategory": "none"
    })

ann_id = 1
image_id = 1

for seq in sequences:
    seq_img_dir = dataset_root / split / seq / "img1"
    seq_gt_path = dataset_root / split / seq / "gt" / "gt.txt"

    # Read the ground truth file
    df = pd.read_csv(seq_gt_path, header=None)
    df.columns = ['frame','id','left','top','width','height','conf','class','visibility']

    # Process each image in the sequence
    for img_file in sorted(os.listdir(seq_img_dir)):
        try:
            frame_number = int(os.path.splitext(img_file)[0])
        except Exception as e:
            print(f"Skipping {img_file}: {e}")
            continue

        # Open image to get width and height
        img_path = seq_img_dir / img_file
        try:
            with Image.open(img_path) as img:
                width, height = img.size
        except Exception as e:
            print(f"Error reading {img_path}: {e}")
            continue

        # Add image entry (store relative path starting from the split folder)
        coco_dict["images"].append({
            "id": image_id,
            "file_name": os.path.join(seq, "img1", img_file),
            "width": width,
            "height": height
        })

        # Filter annotations for the current frame
        df_frame = df[df["frame"] == frame_number]
        for _, row in df_frame.iterrows():
            bbox = [
                int(row['left']),
                int(row['top']),
                int(row['width']),
                int(row['height'])
            ]
            coco_dict["annotations"].append({
                "id": ann_id,
                "image_id": image_id,
                "category_id": int(row['class']),
                "bbox": bbox,
                "area": int(row['width']) * int(row['height']),
                "iscrowd": 0,
            })
            ann_id += 1

        image_id += 1

# Save the converted COCO JSON
annotations_path = data_dir / "brackishmot_coco.json"
with open(annotations_path, "w") as f:
    json.dump(coco_dict, f)

print(f"COCO annotations saved to: {annotations_path}")


In [ ]:
import supervision as sv
from pathlib import Path
import random
import matplotlib.pyplot as plt

# Define the images directory (adjust this if your structure is different)
# Assuming dataset_root is already defined as: data_dir / "BrackishMOT"
images_directory_path = dataset_root / "train"  # This should contain your sequence folders (e.g., brackishMOT-97, etc.)

# Load the dataset using the COCO annotations we just created
dataset = sv.DetectionDataset.from_coco(
    images_directory_path=str(images_directory_path),
    annotations_path=str(annotations_path),
)

print(f"Loaded dataset with {len(dataset)} images.")
print(f"Dataset classes: {dataset.classes}")

# Initialize annotators for visualization
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

# Visualize a grid of random samples from the dataset
annotated_images = []
image_example = None

for _ in range(16):
    idx = random.randint(0, len(dataset) - 1)
    _, image, annotations = dataset[idx]

    # Map class IDs to their label names
    ann_labels = [dataset.classes[class_id] for class_id in annotations.class_id]

    annotated_image = image.copy()
    annotated_image = box_annotator.annotate(annotated_image, annotations)
    annotated_image = label_annotator.annotate(annotated_image, annotations, ann_labels)
    annotated_images.append(annotated_image)

    if len(annotations) > 0:
        image_example = annotated_image

# Plot the grid of annotated images
sv.plot_images_grid(
    annotated_images,
    grid_size=(4, 4),
    titles=None,
    size=(80, 48),
    cmap="gray"
)

# Optionally, save one preview image
output_preview_dir = Path("data_preview")
output_preview_dir.mkdir(exist_ok=True, parents=True)
plt.imsave(output_preview_dir / "brackishmot_sample_image.png", image_example)

print("Visualization complete!")


In [ ]:
print(f"Dataset classes: {dataset.classes}")

# DATASET STATISTICS - Calculate exact counts

In [ ]:
import json
import os
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

def analyze_dataset_statistics(dataset_root, annotations_path, split="train", sequences=None):
    """
    Analyzes the BrackishMOT dataset to extract comprehensive statistics

    Args:
        dataset_root: Path to the dataset root directory
        annotations_path: Path to the COCO annotations JSON file
        split: Dataset split to analyze (default: "train")
        sequences: List of sequence names to analyze (if None, all sequences will be analyzed)

    Returns:
        Dictionary containing dataset statistics
    """
    stats = {}

    # 1. Load COCO annotations file
    print(f"Loading annotations from {annotations_path}...")
    with open(annotations_path, 'r') as f:
        coco_data = json.load(f)

    # 2. Basic statistics from COCO annotations
    stats['total_images'] = len(coco_data['images'])
    stats['total_annotations'] = len(coco_data['annotations'])
    stats['total_categories'] = len(coco_data['categories'])

    # 3. Categories
    stats['categories'] = [cat['name'] for cat in coco_data['categories']]

    # 4. Per-category statistics
    category_map = {cat['id']: cat['name'] for cat in coco_data['categories']}
    category_counts = Counter([cat_id for ann in coco_data['annotations'] for cat_id in [ann['category_id']]])
    stats['annotations_per_category'] = {category_map[cat_id]: count for cat_id, count in category_counts.items()}

    # 5. Images per sequence
    if not sequences:
        # Extract sequence names from image file_name paths
        all_file_names = [img['file_name'] for img in coco_data['images']]
        sequences = list(set([file_name.split('/')[0] for file_name in all_file_names]))

    sequence_counts = Counter([img['file_name'].split('/')[0] for img in coco_data['images']])
    stats['images_per_sequence'] = {seq: sequence_counts.get(seq, 0) for seq in sequences}

    # 6. Check image channels
    # Sample a few images to detect channels
    sample_image_paths = []
    for seq in sequences:
        seq_img_dir = os.path.join(dataset_root, split, seq, "img1")
        if os.path.exists(seq_img_dir):
            img_files = os.listdir(seq_img_dir)
            if img_files:
                sample_image_paths.append(os.path.join(dataset_root, split, seq, "img1", img_files[0]))
                break

    if sample_image_paths:
        sample_img = Image.open(sample_image_paths[0])
        if sample_img.mode == 'RGB':
            stats['channels'] = 3
            stats['color_mode'] = 'RGB'
        elif sample_img.mode == 'L':
            stats['channels'] = 1
            stats['color_mode'] = 'Grayscale'
        else:
            stats['channels'] = len(sample_img.getbands())
            stats['color_mode'] = sample_img.mode
    else:
        stats['channels'] = "Unknown (no sample images found)"
        stats['color_mode'] = "Unknown"

    # 7. Raw metadata format analysis
    raw_formats = []
    for seq in sequences:
        gt_path = os.path.join(dataset_root, split, seq, "gt", "gt.txt")
        if os.path.exists(gt_path):
            with open(gt_path, 'r') as f:
                first_line = f.readline().strip()
                # Check if CSV format
                delimiter = ',' if ',' in first_line else ' '
                column_count = len(first_line.split(delimiter))
                raw_formats.append(f"{seq}: {column_count} columns with delimiter '{delimiter}'")

    stats['raw_format_info'] = raw_formats

    # 8. Additional statistics
    # Calculate annotations per image
    image_ids = [ann['image_id'] for ann in coco_data['annotations']]
    annotations_per_image = Counter(image_ids)
    stats['avg_annotations_per_image'] = sum(annotations_per_image.values()) / len(annotations_per_image)
    stats['max_annotations_per_image'] = max(annotations_per_image.values())
    stats['min_annotations_per_image'] = min(annotations_per_image.values())

    # 9. Generate visualizations
    # Category distribution
    plt.figure(figsize=(12, 6))
    categories = list(stats['annotations_per_category'].keys())
    counts = list(stats['annotations_per_category'].values())
    plt.bar(categories, counts)
    plt.title('Annotations per Category')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('category_distribution.png')

    # Sequence distribution
    plt.figure(figsize=(12, 6))
    sequences = list(stats['images_per_sequence'].keys())
    counts = list(stats['images_per_sequence'].values())
    plt.bar(sequences, counts)
    plt.title('Images per Sequence')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('sequence_distribution.png')

    # Print summary statistics
    print(f"\n{'='*50}")
    print(f"BrackishMOT Dataset Statistics Summary")
    print(f"{'='*50}")
    print(f"Total Images: {stats['total_images']}")
    print(f"Total Annotations: {stats['total_annotations']}")
    print(f"Categories ({stats['total_categories']}): {', '.join(stats['categories'])}")
    print(f"Color Mode: {stats['color_mode']} ({stats['channels']} channels)")
    print(f"Average Annotations per Image: {stats['avg_annotations_per_image']:.2f}")
    print(f"\nAnnotations per Category:")
    for cat, count in stats['annotations_per_category'].items():
        print(f"  - {cat}: {count}")
    print(f"\nImages per Sequence:")
    for seq, count in stats['images_per_sequence'].items():
        print(f"  - {seq}: {count}")
    print(f"\nRaw Format Information:")
    for fmt in stats['raw_format_info']:
        print(f"  - {fmt}")
    print(f"{'='*50}")

    return stats

# Add this at the end of your notebook to calculate and display the statistics
if __name__ == "__main__" or True:  # This will run in Jupyter notebook
    # Define paths (make sure these match your existing paths)
    dataset_root = data_dir / "BrackishMOT"
    annotations_path = data_dir / "brackishmot_coco.json"
    sequences = ['brackishMOT-97','brackishMOT-22','brackishMOT-70',
                 'brackishMOT-07','brackishMOT-53','brackishMOT-42']

    # Run the analysis
    dataset_stats = analyze_dataset_statistics(
        dataset_root=dataset_root,
        annotations_path=annotations_path,
        split="train",
        sequences=sequences
    )

    # You can also save the stats to a JSON file for later reference
    with open(data_dir / "brackishmot_statistics.json", 'w') as f:
        # Convert any non-serializable values to strings
        serializable_stats = {k: (str(v) if not isinstance(v, (dict, list, str, int, float, bool, type(None))) else v)
                             for k, v in dataset_stats.items()}
        json.dump(serializable_stats, f, indent=2)

    print(f"Statistics saved to {data_dir / 'brackishmot_statistics.json'}")